In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
#pd.options.display.max_rows = 100
import seaborn as sns 
import indicoio
import sys
sys.path.append("../../Code")
import utils
sys.path.append("../../Tools/python-client")
from __future__ import print_function
import time
from pprint import pprint
import settings as s
import images as im
import cognitive as cog
import files as f
import profiles
import preprocessing as pre

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Load Country Data 

In [2]:
countries = pd.read_csv('../1/country_info.csv', index_col=0)

In [3]:
countries = countries.rename(columns={'languages': 'country_languages'})

In [4]:
countries.head()

,alpha2Code,alpha3Code,country_languages,borders,capital,gini,latlng,name,nativeName,population,region,regionalBlocs,subregion
0,AF,AFG,"['Pashto', 'Uzbek', 'Turkmen']","['IRN', 'PAK', 'TKM', 'UZB', 'TJK', 'CHN']",Kabul,27.8,"[33.0, 65.0]",Afghanistan,افغانستان,27657145,Asia,['SAARC'],Southern Asia
1,AX,ALA,['Swedish'],[],Mariehamn,NaN,"[60.116667, 19.9]",Åland Islands,Åland,28875,Europe,['EU'],Northern Europe
2,AL,ALB,['Albanian'],"['MNE', 'GRC', 'MKD', 'KOS']",Tirana,34.5,"[41.0, 20.0]",Albania,Shqipëria,2886026,Europe,['CEFTA'],Southern Europe
3,DZ,DZA,['Arabic'],"['TUN', 'LBY', 'NER', 'ESH', 'MRT', 'MLI', 'MAR']",Algiers,35.3,"[28.0, 3.0]",Algeria,الجزائر,40400000,Africa,"['AU', 'AL']",Northern Africa
4,AS,ASM,"['English', 'Samoan']",[],Pago Pago,NaN,"[-14.33333333, -170.0]",American Samoa,American Samoa,57100,Oceania,[],Polynesia


In [5]:
country_cols = ['alpha2Code', 'region', 'regionalBlocs', 'subregion', 'latlng', 'borders', 'country_languages']

In [6]:
def choose_match_name(cnty_tbl, cult_table):
    names = []
    corrections = {
        "Russian Federation": "Russia",
        "Sierra Leone": "Sierra Leon",
        "Korea (Republic of)": "South Korea",
        "Iran (Islamic Republic of)": "Iran"
    }
    for _, r in cnty_tbl.iterrows():
        if r['nativeName'] in cult_table['country'].unique():
            names.append(r['nativeName'])
        elif r['name'] in corrections.keys():
            names.append(corrections[r['name']])
        else:
            names.append(r['name'])
    return names

In [7]:
cultural_factors = pd.read_csv('../../Tools/world_cultural_factors.csv')
cultural_factor_cols = ['country', 'selected_for', 'power_distance', 'individualism', 'masculinity', 'uncertainty_avoidance']

In [8]:
countries['matchName'] = choose_match_name(countries, cultural_factors)

In [9]:
cntry_cult_tbl = countries.join(cultural_factors[cultural_factor_cols].set_index('country'), on='matchName').drop_duplicates(subset="alpha2Code", keep="first")

In [10]:
cntry_cult_tbl[~cntry_cult_tbl.power_distance.isnull()].head(2)

,alpha2Code,alpha3Code,country_languages,borders,capital,gini,latlng,name,nativeName,population,region,regionalBlocs,subregion,matchName,selected_for,power_distance,individualism,masculinity,uncertainty_avoidance
10,AR,ARG,"['Spanish', 'Guaraní']","['BOL', 'BRA', 'CHL', 'PRY', 'URY']",Buenos Aires,44.5,"[-34.0, -64.0]",Argentina,Argentina,43590400,Americas,['USAN'],South America,Argentina,addition,49.0,46.0,56.0,86.0
13,AU,AUS,['English'],[],Canberra,30.5,"[-27.0, 133.0]",Australia,Australia,24117360,Oceania,[],Australia and New Zealand,Australia,High individualisim,36.0,90.0,61.0,51.0


# Load Review, Host , Guest Tables

### Listings 

In [11]:
listings = pd.read_csv('../2/nr_restrict_listings.csv', index_col=0)
full_listings = listings[listings['room_type'] == 'Entire home/apt']
shared_listings = listings[listings['room_type'] != 'Entire home/apt']

In [12]:
len(listings)

6924

### Guests

In [13]:
# Man guests because we are gaining information about hosts guests - not looking at reviews

In [14]:
guests = pd.read_csv('../2/man_guests.csv', low_memory=False, index_col=0)

In [15]:
guests = guests.join(countries[country_cols].set_index('alpha2Code'), on='ccode')

In [16]:
guests = guests.drop_duplicates(subset="id", keep="last")

In [17]:
len(guests)

181376

In [18]:
guests_cols = ['id', 'ccode', 'region', 'subregion', 'country_languages']

### Hosts

In [19]:
hosts = pd.read_csv('../2/nr_restrict_hosts.csv', index_col=0)
hosts = hosts[~hosts.index.isnull()]

In [20]:
len(hosts)

6924

### Reviews

In [21]:
reviews = pd.read_csv('../2/nr_restrict_reviews.csv', index_col=0)

### Reduce reviews to those where we have both hosts and guests 

In [22]:
reviews_match, h_ids, g_ids = utils.reduce_reviews(reviews, hosts, guests)

Total Reviews: 105600 
Unique Hosts: 5591 
Unique Guests: 96255


In [23]:
results_table = reviews_match[['id', 'date', 'reviewer_id', 'listing_id', 'recipient_id']].reset_index(drop=True)

### Get Trip table 

In [24]:
hosts_trip = pd.read_csv('../2/man_hostTrips.csv', index_col=0)

In [36]:
len(hosts_trip.hostId.unique())

3587

In [25]:
host_trip_summary = profiles.summarize_host_table(hosts_trip, countries)

In [30]:
host_trip_summary.head()

,countries_visited,id,regions_visited,sub_regions_visited
0,"{'HU': 2, 'CZ': 1, 'AT': 1}",10001390,{'Europe': 4},"{'Eastern Europe': 3, 'Western Europe': 1}"
1,"{'DE': 1, 'AR': 1, 'MX': 2, 'CA': 1, 'US': 1, ...",1000278,"{'Europe': 1, 'Americas': 5, 'Asia': 1}","{'Western Europe': 1, 'South America': 1, 'Cen..."
2,"{'NL': 1, 'DE': 1, 'US': 4}",10003254,"{'Europe': 2, 'Americas': 4}","{'Western Europe': 2, 'Northern America': 4}"
3,"{'US': 1, 'JE': 1}",10013093,"{'Americas': 1, 'Europe': 1}","{'Northern America': 1, 'Northern Europe': 1}"
4,{'PE': 2},10015933,{'Americas': 2},{'South America': 2}


### Get Guest Review Table 

In [31]:
hg_reviews = pd.read_csv('../2/man_hostReviews.csv', low_memory=False, index_col=0)

#### Cleaning up Reviews

#### Getting Count Tables

In [32]:
reviews_per_guest = hg_reviews[['recipient_id', 'total_host_reviews']].astype(int).groupby('recipient_id').mean()

In [33]:
reviews_from_host = hg_reviews[~hg_reviews['reviewer_id'].isnull()][['reviewer_id', 'total_host_reviews']].astype(int).groupby('reviewer_id').count()

## Get information about where Host has visited 

In [34]:
import geopy.distance

In [35]:
def get_num_countries(x, htrips):
    if x in set(htrips.id):
        return len(htrips[htrips.id == x]['countries_visited'].keys())
    else:
        return 0

In [37]:
len(results_table)

105600

In [38]:
results_table = results_table.join(host_trip_summary.set_index('id'), on='recipient_id')

In [40]:
results_table['h_min_used'] = [np.sum(list(v.values())) if v not in [None, "NaN", np.nan] else 0 for v in results_table['regions_visited']]

In [41]:
results_table['h_min_cntry'] = [len(v.keys()) if v not in [None, "NaN", np.nan] else 0 for v in results_table['countries_visited']]

In [42]:
results_table['h_min_regions'] = [len(v.keys()) if v not in [None, "NaN", np.nan] else 0 for v in results_table['regions_visited']]

In [43]:
results_table['h_min_subregions'] = [len(v.keys()) if v not in [None, "NaN", np.nan] else 0 for v in results_table['sub_regions_visited']]

In [44]:
results_table = results_table.join(guests[guests_cols].set_index('id'), on='reviewer_id')

In [45]:
results_table['eng_in_guest_clangs'] = [1 if v not in [np.nan, None] and "English" in set(eval(v)) else 0 for v in results_table['country_languages']]

In [46]:
def get_h_visit_g(val, opts):
    if opts in [np.nan, None, "NaN"]:
        return 0
    else:
        if val in opts.keys():
            return 1
        else:
            return 0

In [47]:
results_table['h_visit_g_cntry'] =  [get_h_visit_g(r['ccode'] , r['countries_visited']) for _, r in results_table.iterrows()]

In [48]:
results_table['h_visit_g_region'] =  [get_h_visit_g(r['region'] , r['regions_visited']) for _, r in results_table.iterrows()]

In [49]:
results_table['h_visit_g_sregion'] =  [get_h_visit_g(r['subregion'] , r['sub_regions_visited']) for _, r in results_table.iterrows()]

In [50]:
results_table['same_country'] = results_table.ccode.map(lambda x: 1 if x == s.HOSTS_CCODE else 0)

In [52]:
results_table['same_region'] = results_table.region.map(lambda x: 1 if x == s.HOSTS_REGION else 0)

In [53]:
results_table['same_subregion'] = results_table.subregion.map(lambda x: 1 if x == s.HOSTS_SUBREGION else 0)

In [54]:
results_table =results_table.join(cntry_cult_tbl[['alpha2Code', 'power_distance', 'individualism', 'masculinity', 'uncertainty_avoidance']].set_index('alpha2Code'), on= 'ccode')

In [56]:
results_table[results_table['individualism'].isnull()]

,id,date,reviewer_id,listing_id,recipient_id,countries_visited,regions_visited,sub_regions_visited,h_min_used,h_min_cntry,h_min_regions,h_min_subregions,ccode,region,subregion,country_languages,eng_in_guest_clangs,h_visit_g_cntry,h_visit_g_region,h_visit_g_sregion,same_country,same_region,same_subregion,power_distance,individualism,masculinity,uncertainty_avoidance
37,93343,2010-09-08,207840,5099,7322,NaN,NaN,NaN,0,0,0,0,TT,Americas,Caribbean,['English'],1,0,0,0,0,1,0,NaN,NaN,NaN,NaN
101,645023,2011-10-19,1119625,5172,7386,"{'CO': 1, 'US': 3, 'IT': 3}","{'Americas': 4, 'Europe': 3}","{'South America': 1, 'Northern America': 3, 'S...",7,3,2,3,BA,Europe,Southern Europe,"['Bosnian', 'Croatian', 'Serbian']",0,0,1,1,0,0,0,NaN,NaN,NaN,NaN
140,8211,2009-08-24,32936,5178,8967,NaN,NaN,NaN,0,0,0,0,HN,Americas,Central America,['Spanish'],0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
159,89441,2010-09-03,144158,5178,8967,NaN,NaN,NaN,0,0,0,0,CL,Americas,South America,['Spanish'],0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
168,153602,2010-12-14,182487,5178,8967,NaN,NaN,NaN,0,0,0,0,AW,Americas,Caribbean,"['Dutch', '(Eastern) Punjabi']",0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
170,155771,2010-12-19,11327,5178,8967,NaN,NaN,NaN,0,0,0,0,JE,Europe,Northern Europe,"['English', 'French']",1,0,0,0,0,0,0,NaN,NaN,NaN,NaN
187,222545,2011-04-11,327452,5178,8967,NaN,NaN,NaN,0,0,0,0,CL,Americas,South America,['Spanish'],0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
190,263586,2011-05-15,299564,5178,8967,NaN,NaN,NaN,0,0,0,0,HR,Europe,Southern Europe,['Croatian'],0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
208,740991,2011-11-24,721906,5178,8967,NaN,NaN,NaN,0,0,0,0,ZA,Africa,Southern Africa,"['Afrikaans', 'English', 'Southern Ndebele', '...",1,0,0,0,0,0,0,NaN,NaN,NaN,NaN
295,18234728,2014-08-24,6213375,5178,8967,NaN,NaN,NaN,0,0,0,0,CZ,Europe,Eastern Europe,"['Czech', 'Slovak']",0,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [57]:
h_power = int(cntry_cult_tbl[cntry_cult_tbl.alpha2Code == s.HOSTS_CCODE]['power_distance'])

In [58]:
h_individualism = int(cntry_cult_tbl[cntry_cult_tbl.alpha2Code == s.HOSTS_CCODE]['individualism'])

In [59]:
h_masculinity = int(cntry_cult_tbl[cntry_cult_tbl.alpha2Code == s.HOSTS_CCODE]['masculinity'])
h_uncertainty_avoid = int(cntry_cult_tbl[cntry_cult_tbl.alpha2Code == s.HOSTS_CCODE]['uncertainty_avoidance'])

In [60]:
def get_cultural_difference(tbl, col, hval):
    vals = []
    for _, r in tbl.iterrows():
        if r[col] in [np.nan, None]:
            vals.append(np.nan)
        else:
            vals.append(hval-r[col])
    return vals

In [61]:
results_table['power_difference'] = get_cultural_difference(results_table, 'power_distance', h_power)                       

In [62]:
results_table['individualism_difference']  = get_cultural_difference(results_table, 'individualism', h_individualism)


In [63]:
results_table['masculanity_difference']  = get_cultural_difference(results_table, 'masculinity', h_masculinity)

In [64]:
results_table['uncertainty_avoid_difference']  = get_cultural_difference(results_table, 'uncertainty_avoidance', h_uncertainty_avoid)

In [65]:
results_table = results_table.drop(["country_languages",'countries_visited', 'regions_visited', 'sub_regions_visited', 'region', 'subregion', 'country_languages', 'power_distance','individualism','masculinity', 'uncertainty_avoidance'], axis=1)



In [66]:
def get_host_guest_countries(src_tbl, host):
    guest_countries = src_tbl[src_tbl['recipient_id'] == host]['ccode'] 
    country_set = {c for c in guest_countries}
    return len(country_set)/len(guest_countries) if len(guest_countries) > 0  else 0

In [67]:
hosts['diverse_guests'] = [get_host_guest_countries(results_table, h) for h in hosts['id']]

In [68]:
results_table = results_table.join(hosts[['id', 'diverse_guests']].set_index('id'), on='recipient_id')

In [69]:
results_table

,id,date,reviewer_id,listing_id,recipient_id,h_min_used,h_min_cntry,h_min_regions,h_min_subregions,ccode,eng_in_guest_clangs,h_visit_g_cntry,h_visit_g_region,h_visit_g_sregion,same_country,same_region,same_subregion,power_difference,individualism_difference,masculanity_difference,uncertainty_avoid_difference,diverse_guests
0,22938,2010-01-03,61471,4989,7118,5,3,2,2,AU,1,0,0,0,0,0,0,4.0,1.0,1.0,-5.0,0.375000
1,234213,2011-04-21,446902,4989,7118,5,3,2,2,AU,1,0,0,0,0,0,0,4.0,1.0,1.0,-5.0,0.375000
2,264416,2011-05-15,540879,4989,7118,5,3,2,2,CA,1,1,1,1,0,1,1,1.0,11.0,10.0,-2.0,0.375000
3,843520,2012-01-06,1357796,4989,7118,5,3,2,2,AU,1,0,0,0,0,0,0,4.0,1.0,1.0,-5.0,0.375000
4,993034,2012-03-12,1757830,4989,7118,5,3,2,2,US,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.375000
5,1119849,2012-04-12,2019278,4989,7118,5,3,2,2,NL,0,0,1,0,0,0,0,2.0,11.0,48.0,-7.0,0.375000
6,1129816,2012-04-14,1857814,4989,7118,5,3,2,2,US,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.375000
7,1292160,2012-05-16,161198,4989,7118,5,3,2,2,US,1,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.375000
8,1624497,2012-07-03,1810355,4989,7118,5,3,2,2,CA,1,1,1,1,0,1,1,1.0,11.0,10.0,-2.0,0.375000
9,4737934,2013-05-22,6043058,4989,7118,5,3,2,2,SE,0,0,1,0,0,0,0,9.0,20.0,57.0,17.0,0.375000


In [70]:
results_table.to_csv("country_cultural_diff.csv")

# By Regions

In [72]:
reviews_match = results_table.copy()

In [73]:
satisfaction = pd.read_csv('../3/satisfaction_scores.csv', index_col=0)

In [76]:
reviews_match = reviews_match.join(satisfaction[['id', 'satisfaction']].set_index('id'), on='id')

In [77]:
profiles.print_country_table(reviews_match, guests, 'region')

,average,guest_count,region,review_count,stddev
0,0.681206,25245,Europe,26533,0.110511
1,0.669611,57257,Americas,64533,0.111642
2,0.664364,5531,Asia,5899,0.116227
3,0.669345,783,Africa,837,0.114803
4,0.683423,7438,Oceania,7796,0.111486
5,0.718636,1,Polar,2,0.004996


In [79]:
profiles.print_country_table_simple(reviews_match, guests, 'region', full_listings)

,average,guest_count,region,review_count,stddev
0,0.666828,36981,Same,41318,0.112835
1,0.672906,23532,Different,24701,0.115571


In [80]:
profiles.print_country_table_simple(reviews_match, guests, 'region', shared_listings)

,average,guest_count,region,review_count,stddev
0,0.674564,21244,Same,23215,0.109313
1,0.688117,15762,Different,16366,0.105188


In [81]:
profiles.print_country_table(reviews_match, guests, 'subregion')

,average,guest_count,review_count,stddev,subregion
0,0.681281,10778,11340,0.110358,Western Europe
1,0.669179,52887,59926,0.111360,Northern America
2,0.676938,3243,3381,0.114108,South America
3,0.673279,3259,3380,0.118701,Southern Europe
4,0.684683,9826,10369,0.106888,Northern Europe
5,0.674214,1382,1444,0.116000,Eastern Europe
6,0.662641,2494,2654,0.114960,Eastern Asia
7,0.671204,852,914,0.117577,Central America
8,0.672115,1331,1423,0.116553,Western Asia
9,0.663358,433,469,0.118084,Southern Asia


In [82]:
profiles.print_country_table_simple(reviews_match, guests, 'subregion', full_listings)

,average,guest_count,review_count,stddev,subregion
0,0.666786,34464,38674,0.112425,Same
1,0.672377,26049,27345,0.115885,Different


In [83]:
profiles.print_country_table_simple(reviews_match, guests, 'subregion', shared_listings)

,average,guest_count,review_count,stddev,subregion
0,0.673533,19351,21252,0.109263,Same
1,0.687861,17655,18329,0.105630,Different
